### Notebook link on databricks :
#### https://databricks-prod-cloudfront.cloud.databricks.com/public/4027ec902e239c93eaaa8714f173bcfc/6487540177752058/1445996215453659/2931679249721832/latest.html

In [ ]:
import pandas as pd
from pyspark.sql import functions as F

steam = spark.read.json('s3://full-stack-bigdata-datasets/Big_Data/Project_Steam/steam_game_output.json')

# First data visualisation

In [ ]:
steam.show()

In [ ]:
steam.select('data').take(1)

In [ ]:
print(steam.count())
print(steam.select('id').distinct().count())
print(steam.select('data.appid').distinct().count())
print(steam.filter(F.col('data.appid') != F.col('id')).count())

In [ ]:
#"id" initial column contains the same info that data["appid"]. We can dive one level into the nested data frame and make our analysis on this.

steam_2 = steam.select('data')
steam_2.printSchema()

In [ ]:
steam_2.show()

## Macro level analysis

In [ ]:
#1 Which publisher has released the most games on Steam?
#  Grouping by publisher and counting occurences.

publisher_occurence = steam_2.groupBy('data.publisher').count().orderBy('count',ascending=False)
display(publisher_occurence.take(20))

## Bar

In [ ]:
#2 What are the best rated games ?
#  Absolute count of positive rates per game

best_rated_games_abs = steam_2.orderBy("data.positive",ascending=False).select("data.name","data.positive")
display(best_rated_games_abs.take(20))

In [ ]:
#  Proportionally most liked games (among rated + 100000 times)

preprocess_rating = steam_2 \
    .withColumn("int_positive", F.col("data.positive").cast("int")) \
    .withColumn("int_negative", F.col("data.negative").cast("int")) \
    .withColumn("rating_ratio", F.col("int_positive")/(F.col("int_positive")+F.col("int_negative"))) \
    .select("data.name","int_positive","int_negative","rating_ratio")
preprocess_rating.show(5)

best_rated_games_prop = preprocess_rating \
    .filter((F.col("int_positive")+F.col("int_negative")) > 100_000) \
    .orderBy("rating_ratio", ascending=False) \
    .select("name", "rating_ratio")

display(best_rated_games_prop.take(20))

## Bar

In [ ]:
#3 Which years with more releases? Releases during COVID (2020) ?
#  Extracting year from original dataset and counting occurences.

release_year = steam_2.withColumn("release_year", F.substring(F.col("data.release_date"), 1, 4)).select("release_year")

release_year_vis = release_year.groupBy('release_year').count().orderBy('count',ascending=False)
display(release_year_vis.take(10))

## Bar

In [ ]:
#4 How are the prizes distributed? Are there many games with a discount?
#  Groupby price and count occurences // count games with a discount

preprocess_price = steam_2 \
    .withColumn("price_float", F.col("data.initialprice").cast("int")) \
    .withColumn("discount_int", F.col("data.discount").cast("int")) \
    .select("data.name","price_float","discount_int")
preprocess_price.show(3)

games_with_discount = preprocess_price.filter(F.col("discount_int")> 0).count()
print(games_with_discount)

price_distrib = preprocess_price.groupBy('price_float').count().orderBy('count',ascending=False)
display(price_distrib.take(20))

## Pie

In [ ]:
#5 What are the most represented languages?
#  Explode languages for each game and count occurences

language_count = steam_2 \
    .withColumn("languages_array", F.split("data.languages", ", ")) \
    .select("data.name", "languages_array")
language_count.show(5)

exploded_languages = language_count.withColumn("language", F.explode(language_count["languages_array"]))
languages_vis = exploded_languages.groupBy("language").count().orderBy('count',ascending=False)
display(languages_vis.take(20))

## Bar

In [ ]:
#6 How many games prohibited for children under 16/18?
#  Delete non usable info, converting to int, filter to more than 16yrs required age

values_to_delete = ["21+", "7+","MA 15+"]
prep_prohibited_games = steam_2.filter(~F.col("data.required_age").isin(values_to_delete))
print(prep_prohibited_games.count())

prep_prohibited_games = prep_prohibited_games \
    .withColumn("int_required_age", F.col("data.required_age").cast("int")) \
    .filter(F.col("int_required_age") >= 16) \
    .select("int_required_age")

print("amount of games prohibited for children under 16/18 or +"prep_prohibited_games.count()+1)

## Genres analysis

In [ ]:
#1 What are the most represented genres?
#  Split and explode data.genre, count distinct values.

genre_count = steam_2.withColumn("genre_array", F.split("data.genre", ", "))

exploded_genre = genre_count.withColumn("genre", F.explode(genre_count["genre_array"]))
print(exploded_genre.show(5))
print(exploded_genre.select("genre").distinct().count())
exploded_genre_vis = exploded_genre.groupBy("genre").count().orderBy("count",ascending=False)
display(exploded_genre_vis)

## Area

In [ ]:
#2 Are there any genres that have a better positive/negative review ratio?
#  Creating feature rating_ratio on exploded "genre" dataset. 

preprocess_genre_rating = exploded_genre \
    .withColumn("int_positive", F.col("data.positive").cast("int")) \
    .withColumn("int_negative", F.col("data.negative").cast("int")) \
    .withColumn("rating_ratio", F.col("int_positive")/(F.col("int_positive")+F.col("int_negative"))) \
    .select("genre","int_positive","int_negative","rating_ratio")
preprocess_genre_rating.show(5)
print(preprocess_genre_rating.count())

In [ ]:
# Calculating mean for each genre.

rating_ratio_per_genre = preprocess_genre_rating\
    .groupBy("genre")\
    .agg(F.mean("rating_ratio").alias("mean_rating_ratio"))\
    .orderBy("mean_rating_ratio", ascending = False)

display(rating_ratio_per_genre.take(30))

## Bar

In [ ]:
#3 Do some publishers have favorite genres?
#  Focus top 20 publishers. Count genre games per publisher.

main_publishers = publisher_occurence.take(20)
main_publishers_list = [row.publisher for row in main_publishers if row.publisher]
print(main_publishers_list)

publisher_filtered = exploded_genre.filter(F.col("data.publisher").isin(main_publishers_list))

publisher_genre_counts = publisher_filtered.groupBy("data.publisher", "genre") \
                           .agg(F.count(F.lit(1)).alias("genre_per_publisher_count"))
publisher_genre_counts.show(5)

publisher_genre_vis = publisher_genre_counts.orderBy("genre_per_publisher_count", ascending=False)
display(publisher_genre_vis)
    
## Bar (groupby)

In [ ]:
#4 What are the most lucrative genres?
#  Estimate purchase with mean of owners.

luc_genre = exploded_genre.withColumn("owners_array", F.split("data.owners", " .. "))

luc_genre = luc_genre\
    .withColumn("mean_owners",
    (   F.regexp_replace(F.col("owners_array")[0], ",", "").cast("int") +
        F.regexp_replace(F.col("owners_array")[1], ",", "").cast("int")) / 2) \
    .withColumn("price_float", F.col("data.price").cast("int")) \
    .withColumn("revenue_estim",(F.col("price_float") * F.col("mean_owners")))
luc_genre.show(3)

luc_genre_vis = luc_genre.groupBy("genre").agg(F.sum("revenue_estim").alias("total_revenue_estim"))
display(luc_genre_vis)

## Pie

## Platform analysis

In [ ]:
#1 Are most games available on Windows/Mac/Linux instead?
#  Count occurences where the plateform contains a value true (=1)
platform_counts = steam_2.agg(
    F.sum(F.col("data.platforms.linux").cast("int")).alias("linux_count"),
    F.sum(F.col("data.platforms.mac").cast("int")).alias("mac_count"),
    F.sum(F.col("data.platforms.windows").cast("int")).alias("windows_count")
)

display(platform_counts)

## Hist

In [ ]:
#2 Do certain genres tend to be preferentially available on certain platforms?
#  From exploded_genre data, count occurences where the plateform contains a value true (=1)

platform_genre_counts = exploded_genre.groupBy("genre").agg(
    F.sum(F.col("data.platforms.linux").cast("int")).alias("linux_count"),
    F.sum(F.col("data.platforms.mac").cast("int")).alias("mac_count"),
    F.sum(F.col("data.platforms.windows").cast("int")).alias("windows_count"),
    F.count("*").alias("total_count")
)

platform_genre_counts = platform_genre_counts \
    .withColumn("linux_proportion",F.col("linux_count") / F.col("total_count")) \
    .withColumn("mac_proportion",F.col("mac_count") / F.col("total_count")) \
    .withColumn("windows_proportion",F.col("windows_count") / F.col("total_count"))


platform_genre_vis = platform_genre_counts.select("genre", "linux_proportion", "mac_proportion", "windows_proportion")

display(platform_genre_vis)

## Combo